In [1]:
# input = """Register A: 729
# Register B: 0
# Register C: 0

# Program: 0,1,5,4,3,0"""

# input = """Register A: 2024
# Register B: 0
# Register C: 0

# Program: 0,3,5,4,3,0"""


# input = """Register A: 117440
# Register B: 0
# Register C: 0

# Program: 0,3,5,4,3,0"""

input = open("inputs/17").read()

In [2]:
def parse_input(input_str):
    lines = [l for l in input_str.split("\n") if l.strip()]

    registers = []
    for line in lines[:3]:
        value = int(line.split(": ")[1])
        registers.append(value)

    program = lines[3].split(": ")[1]
    program = [int(x) for x in program.split(",")]

    return (registers, program)


registers, program = parse_input(input)

registers, program

([48744869, 0, 0], [2, 4, 1, 2, 7, 5, 1, 3, 4, 4, 5, 5, 0, 3, 3, 0])

In [3]:
len(program)

16

In [4]:
A = 0
B = 1
C = 2


def simulate(registers, program):
    def get_operand_value(operand, literal=True):
        if literal:
            return operand

        if operand <= 3:
            return operand
        elif operand == 7:
            raise Exception("Operand 7 is not a valid operand")
        else:
            return registers[operand % 4]

    outs = []
    program_counter = 0

    while program_counter < len(program) - 1:
        instruction = program[program_counter]
        operand = program[program_counter + 1]
        to_progress = 2

        # print(instruction, operand, registers)

        if instruction == 0:
            # adv
            registers[A] //= 2 ** get_operand_value(operand, literal=False)
        elif instruction == 1:
            # bxl
            registers[B] ^= get_operand_value(operand, literal=True)
        elif instruction == 2:
            # bst
            registers[B] = get_operand_value(operand, literal=False) % 8
            pass
        elif instruction == 3:
            # jnz
            if registers[A] != 0:
                # print("jumping")
                program_counter = get_operand_value(operand, literal=True)
                to_progress = 0
            # else:
            #     print("passing")
        elif instruction == 4:
            # bxc
            # ignores operand
            registers[B] ^= registers[C]
        elif instruction == 5:
            # out
            out = get_operand_value(operand, literal=False) % 8
            outs.append(out)
        elif instruction == 6:
            # bdv
            registers[B] = registers[A] // 2 ** get_operand_value(
                operand, literal=False
            )
        elif instruction == 7:
            # cdv
            registers[C] = registers[A] // 2 ** get_operand_value(
                operand, literal=False
            )
        else:
            raise Exception(f"Unknown instruction: {instruction}")

        program_counter += to_progress

    return tuple(outs)

In [5]:
tuple(program)

(2, 4, 1, 2, 7, 5, 1, 3, 4, 4, 5, 5, 0, 3, 3, 0)

In [6]:
import pandas as pd
from tqdm import tqdm
import itertools

import math


def sample_log_uniform_integers_oom(low_oom: int, high_oom: int, k: int):
    """
    Sample k integers uniformly in log space between two orders of magnitude.

    Args:
        low_oom (int): The lower order of magnitude (e.g., 13 for 10^13).
        high_oom (int): The upper order of magnitude (e.g., 15 for 10^15).
        k (int): The number of integers to sample.

    Returns:
        List[int]: A list of k integers sampled uniformly in log space.
    """
    if low_oom <= 0 or high_oom <= 0:
        raise ValueError("Orders of magnitude must be positive.")
    if low_oom >= high_oom:
        raise ValueError("Lower OOM must be less than upper OOM.")
    if k < 2:
        raise ValueError("k must be at least 2 to include both bounds.")

    # Compute the actual bounds in base-10 logarithmic scale
    L_low = low_oom * math.log(10)  # log(10^low_oom) = low_oom * log(10)
    L_high = high_oom * math.log(10)  # log(10^high_oom) = high_oom * log(10)

    # Generate k points evenly spaced in log space
    samples = []
    for i in range(k):
        L_i = L_low + i * (L_high - L_low) / (k - 1)  # evenly spaced in log scale
        x_i = round(math.exp(L_i))  # back to original scale, rounded to integer
        samples.append(x_i)

    return samples


def sample_log_uniform_integers(low: int, high: int, k: int):
    """
    Sample k integers uniformly in log space between low and high.

    Args:
        low (int): The lower bound (inclusive).
        high (int): The upper bound (inclusive).
        k (int): The number of integers to sample.

    Returns:
        List[int]: A list of k integers sampled uniformly in log space.
    """
    if low <= 0 or high <= 0:
        raise ValueError("Bounds must be positive integers.")
    if low >= high:
        raise ValueError("Lower bound must be less than upper bound.")
    if k < 2:
        raise ValueError("k must be at least 2 to include both bounds.")

    # Calculate log space bounds
    L_low = math.log(low)
    L_high = math.log(high)

    # Generate k points evenly spaced in log space
    samples = []
    for i in range(k):
        L_i = L_low + i * (L_high - L_low) / (k - 1)  # evenly spaced in log scale
        x_i = round(math.exp(L_i))  # back to original scale, rounded to integer
        samples.append(x_i)

    return samples


# ooms = itertools.chain(
#     range(10_000_000_000_000, 10_000_001_000_000),
#     # range(70_000_000_000_000, 70_000_001_000_000),
#     # range(80_000_000_000_000, 80_000_001_000_000),
#     # range(90_000_000_000_000, 90_000_001_000_000),
#     # range(100_000_000_000_000, 100_000_001_000_000),
#     # range(110_000_000_000_000, 110_000_001_000_000),
#     # range(120_000_000_000_000, 120_000_001_000_000),
#     # range(130_000_000_000_000, 130_000_001_000_000),
#     # range(140_000_000_000_000, 140_000_001_000_000),
#     # range(150_000_000_000_000, 150_000_001_000_000),
#     # range(160_000_000_000_000, 160_000_001_000_000),
#     # range(170_000_000_000_000, 170_000_001_000_000),
#     # range(180_000_000_000_000, 180_000_001_000_000),
#     # range(190_000_000_000_000, 190_000_001_000_000),
#     # range(200_000_000_000_000, 200_000_001_000_000),
#     range(1_000_000_000_000_000, 1_000_000_001_000_000),
# )

# ooms = sample_log_uniform_integers_oom(13, 15, 10_000_000)
# ooms = sample_log_uniform_integers(10**13, 10**15, 10_000_000)
ooms = sample_log_uniform_integers(int(3.7 * 10**13), int(3.9 * 10**13), 10_000_000)

rows = []
for i in tqdm(ooms):
    new_registers = registers.copy()
    new_registers[A] = i

    result = simulate(new_registers, program)
    result_len = len(result)

    rows.append(
        {
            "i": i,
            "result_len": result_len,
            "result": ",".join([str(x) for x in result]),
        }
    )

    if result == tuple(program):
        print(i)
        break

100%|██████████| 10000000/10000000 [03:04<00:00, 54274.63it/s]


In [7]:
df = pd.DataFrame(rows).loc[lambda df: df.result_len == 16]
# df = pd.DataFrame(rows)

In [8]:
df

,i,result_len,result
0,37000000000000,16,"1,1,1,3,7,4,0,2,7,0,1,3,1,3,3,0"
1,37000000194782,16,"2,7,3,1,2,3,1,2,7,0,1,3,1,3,3,0"
2,37000000389564,16,"3,7,7,1,5,1,7,0,7,0,1,3,1,3,3,0"
3,37000000584346,16,"1,3,1,0,3,3,7,0,7,0,1,3,1,3,3,0"
4,37000000779127,16,"5,1,5,3,3,3,7,0,7,0,1,3,1,3,3,0"
...,...,...,...
9999995,38999999178758,16,"7,1,1,4,7,6,3,3,4,1,4,3,7,3,3,0"
9999996,38999999384068,16,"5,1,3,0,1,7,4,3,4,1,4,3,7,3,3,0"
9999997,38999999589379,16,"3,1,5,6,1,4,4,3,4,1,4,3,7,3,3,0"
9999998,38999999794689,16,"0,1,7,7,5,4,3,5,4,1,4,3,7,3,3,0"


In [9]:
df.groupby("result").size().sort_values(ascending=False).head(50)

result
1,4,7,5,1,3,5,3,3,3,1,1,5,4,3,0    3
1,7,6,1,1,5,6,3,1,4,7,2,1,4,3,0    3
3,2,1,5,1,5,1,3,3,7,7,6,1,4,3,0    3
5,0,7,3,7,0,7,1,5,1,3,7,6,3,3,0    3
3,7,4,1,0,0,7,3,7,2,1,1,5,4,3,0    3
1,1,6,3,7,2,1,5,3,5,2,7,6,3,3,0    3
5,3,7,7,3,6,6,3,1,5,1,1,5,4,3,0    3
1,0,7,3,7,0,3,7,5,1,3,7,6,3,3,0    3
3,5,3,3,7,2,1,3,7,1,3,7,6,3,3,0    3
1,0,7,3,7,0,7,7,0,5,2,7,6,3,3,0    3
7,2,1,7,2,1,5,1,7,7,3,5,6,3,3,0    3
1,5,1,7,7,5,1,4,1,2,1,6,5,4,3,0    3
6,2,7,3,7,5,3,3,0,1,3,7,6,3,3,0    3
7,7,1,2,7,2,2,1,1,1,5,7,3,3,3,0    2
1,7,4,1,6,1,3,6,6,3,1,4,5,4,3,0    2
1,1,1,2,1,7,7,5,1,7,3,6,6,3,3,0    2
4,7,2,1,1,1,3,3,5,7,0,3,4,4,3,0    2
1,1,3,1,7,0,7,4,7,6,2,7,6,3,3,0    2
2,1,1,2,7,3,6,1,7,7,0,7,7,4,3,0    2
6,3,7,6,1,1,1,1,4,7,2,1,5,3,3,0    2
1,1,4,7,2,1,7,7,3,7,1,6,3,4,3,0    2
1,7,3,0,1,3,7,3,2,1,5,2,1,3,3,0    2
6,1,1,5,1,1,1,2,7,0,7,3,5,4,3,0    2
1,7,2,5,7,2,1,3,4,7,2,1,5,3,3,0    2
0,1,7,2,1,7,2,1,7,5,1,1,5,4,3,0    2
3,5,1,1,1,1,5,4,3,3,3,1,5,4,3,0    2
7,6,4,7,6,1,6,7,7,0,7,3,1,4,3,0

In [10]:
df.groupby("result_len").size().sort_values(ascending=False)

result_len
16    10000000
dtype: int64

In [11]:
df[df.result_len == 16].groupby("result").size().sort_values(ascending=False)

result
1,4,7,5,1,3,5,3,3,3,1,1,5,4,3,0    3
1,7,6,1,1,5,6,3,1,4,7,2,1,4,3,0    3
3,2,1,5,1,5,1,3,3,7,7,6,1,4,3,0    3
5,0,7,3,7,0,7,1,5,1,3,7,6,3,3,0    3
3,7,4,1,0,0,7,3,7,2,1,1,5,4,3,0    3
                                  ..
2,1,3,4,7,1,6,3,3,1,3,7,6,3,3,0    1
2,1,3,4,7,1,6,5,3,7,0,7,7,4,3,0    1
2,1,3,4,7,1,6,6,3,2,7,2,1,4,3,0    1
2,1,3,4,7,1,6,7,7,2,1,0,7,4,3,0    1
2,1,3,4,7,1,6,1,3,4,7,1,1,4,3,0    1
Length: 9995812, dtype: int64

In [12]:
df["result_last_9"] = df["result"].str.slice(-17)
df["result_last_8"] = df["result"].str.slice(-15)
df["result_last_7"] = df["result"].str.slice(-13)
df["result_last_6"] = df["result"].str.slice(-11)
df["result_last_5"] = df["result"].str.slice(-9)
df["result_last_4"] = df["result"].str.slice(-7)
df["result_last_3"] = df["result"].str.slice(-5)
df["result_last_2"] = df["result"].str.slice(-3)

In [13]:
df["result_first_3"] = df["result"].str.slice(0, 5)
df["result_first_4"] = df["result"].str.slice(0, 7)
df["result_first_5"] = df["result"].str.slice(0, 9)
df["result_first_6"] = df["result"].str.slice(0, 11)
df["result_first_7"] = df["result"].str.slice(0, 13)
df["result_first_8"] = df["result"].str.slice(0, 15)
df["result_first_9"] = df["result"].str.slice(0, 17)

In [14]:
df.groupby("result_first_3").size().sort_values(ascending=False)

result_first_3
7,2,1    104062
1,1,1     98650
7,3,1     96691
1,2,1     95993
1,7,7     86123
          ...  
6,2,5       153
6,6,2       153
6,6,0       151
6,6,6       147
6,6,4       145
Length: 512, dtype: int64

In [15]:
df.groupby("result_first_4").size().sort_values(ascending=False)

result_first_4
7,2,1,1    27567
1,7,3,1    25567
1,7,2,1    23761
4,7,2,1    22760
7,3,1,1    22044
           ...  
7,6,6,6       15
0,6,2,1       15
2,6,6,0       15
0,2,0,7       13
2,6,2,0       12
Length: 4003, dtype: int64

In [16]:
df.groupby("result_first_8").size().sort_values(ascending=False)

result_first_8
1,1,1,4,7,2,1,1    112
1,1,4,7,2,1,1,1    110
7,2,1,1,4,7,2,1    103
1,1,1,1,4,7,2,1    103
1,7,3,1,1,7,2,1    102
                  ... 
7,7,7,7,7,7,6,0      1
7,7,7,7,7,7,6,5      1
7,7,7,7,7,7,6,7      1
7,7,7,7,7,7,4,6      1
7,7,7,7,7,7,4,7      1
Length: 3707491, dtype: int64

In [17]:
8**8

16777216

In [18]:
df.groupby("result_last_8").size().sort_values(ascending=False)

result_last_8
1,7,3,1,5,4,3,0    6034
1,1,1,1,5,4,3,0    5804
7,3,1,1,5,4,3,0    5619
1,1,2,7,7,4,3,0    5352
7,7,6,1,5,3,3,0    5350
                   ... 
5,5,7,7,4,3,3,0      81
1,2,2,1,4,3,3,0      81
1,7,4,3,7,3,3,0      81
4,1,5,3,4,3,3,0      81
4,1,7,1,7,3,3,0      81
Length: 21816, dtype: int64

In [19]:
df[df.result_first_8 == "1,1,4,7,2,1,1,1"]

,i,result_len,result,result_last_9,result_last_8,result_last_7,result_last_6,result_last_5,result_last_4,result_last_3,result_last_2,result_first_3,result_first_4,result_first_5,result_first_6,result_first_7,result_first_8,result_first_9
256029,37049903421186,16,"1,1,4,7,2,1,1,1,1,7,1,3,7,3,3,0","1,1,7,1,3,7,3,3,0","1,7,1,3,7,3,3,0","7,1,3,7,3,3,0","1,3,7,3,3,0","3,7,3,3,0","7,3,3,0","3,3,0","3,0","1,1,4","1,1,4,7","1,1,4,7,2","1,1,4,7,2,1","1,1,4,7,2,1,1","1,1,4,7,2,1,1,1","1,1,4,7,2,1,1,1,1"
300093,37058498860858,16,"1,1,4,7,2,1,1,1,1,7,2,1,7,3,3,0","1,1,7,2,1,7,3,3,0","1,7,2,1,7,3,3,0","7,2,1,7,3,3,0","2,1,7,3,3,0","1,7,3,3,0","7,3,3,0","3,3,0","3,0","1,1,4","1,1,4,7","1,1,4,7,2","1,1,4,7,2,1","1,1,4,7,2,1,1","1,1,4,7,2,1,1,1","1,1,4,7,2,1,1,1,1"
305715,37059595671906,16,"1,1,4,7,2,1,1,1,7,3,1,1,7,3,3,0","1,7,3,1,1,7,3,3,0","7,3,1,1,7,3,3,0","3,1,1,7,3,3,0","1,1,7,3,3,0","1,7,3,3,0","7,3,3,0","3,3,0","3,0","1,1,4","1,1,4,7","1,1,4,7,2","1,1,4,7,2,1","1,1,4,7,2,1,1","1,1,4,7,2,1,1,1","1,1,4,7,2,1,1,1,7"
308491,37060137261158,16,"1,1,4,7,2,1,1,1,7,2,1,1,7,3,3,0","1,7,2,1,1,7,3,3,0","7,2,1,1,7,3,3,0","2,1,1,7,3,3,0","1,1,7,3,3,0","1,7,3,3,0","7,3,3,0","3,3,0","3,0","1,1,4","1,1,4,7","1,1,4,7,2","1,1,4,7,2,1","1,1,4,7,2,1,1","1,1,4,7,2,1,1,1","1,1,4,7,2,1,1,1,7"
312213,37060863424585,16,"1,1,4,7,2,1,1,1,1,7,3,1,7,3,3,0","1,1,7,3,1,7,3,3,0","1,7,3,1,7,3,3,0","7,3,1,7,3,3,0","3,1,7,3,3,0","1,7,3,3,0","7,3,3,0","3,3,0","3,0","1,1,4","1,1,4,7","1,1,4,7,2","1,1,4,7,2,1","1,1,4,7,2,1,1","1,1,4,7,2,1,1,1","1,1,4,7,2,1,1,1,1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9571989,38912223944018,16,"1,1,4,7,2,1,1,1,3,0,2,6,4,3,3,0","1,3,0,2,6,4,3,3,0","3,0,2,6,4,3,3,0","0,2,6,4,3,3,0","2,6,4,3,3,0","6,4,3,3,0","4,3,3,0","3,3,0","3,0","1,1,4","1,1,4,7","1,1,4,7,2","1,1,4,7,2,1","1,1,4,7,2,1,1","1,1,4,7,2,1,1,1","1,1,4,7,2,1,1,1,3"
9689457,38936294528850,16,"1,1,4,7,2,1,1,1,5,1,5,3,4,3,3,0","1,5,1,5,3,4,3,3,0","5,1,5,3,4,3,3,0","1,5,3,4,3,3,0","5,3,4,3,3,0","3,4,3,3,0","4,3,3,0","3,3,0","3,0","1,1,4","1,1,4,7","1,1,4,7,2","1,1,4,7,2,1","1,1,4,7,2,1,1","1,1,4,7,2,1,1,1","1,1,4,7,2,1,1,1,5"
9697317,38937905667346,16,"1,1,4,7,2,1,1,1,5,4,3,3,4,3,3,0","1,5,4,3,3,4,3,3,0","5,4,3,3,4,3,3,0","4,3,3,4,3,3,0","3,3,4,3,3,0","3,4,3,3,0","4,3,3,0","3,3,0","3,0","1,1,4","1,1,4,7","1,1,4,7,2","1,1,4,7,2,1","1,1,4,7,2,1,1","1,1,4,7,2,1,1,1","1,1,4,7,2,1,1,1,5"
9717798,38942104164706,16,"1,1,4,7,2,1,1,1,1,6,7,7,4,3,3,0","1,1,6,7,7,4,3,3,0","1,6,7,7,4,3,3,0","6,7,7,4,3,3,0","7,7,4,3,3,0","7,4,3,3,0","4,3,3,0","3,3,0","3,0","1,1,4","1,1,4,7","1,1,4,7,2","1,1,4,7,2,1","1,1,4,7,2,1,1","1,1,4,7,2,1,1,1","1,1,4,7,2,1,1,1,1"


In [20]:
len("199853403")

9

In [21]:
len("37221260473399") - 6

8

In [22]:
last_7_match = df[df.result_last_7 == ",".join([str(x) for x in program[-7:]])]

for first_k_i in range(4, 10):
    test = last_7_match.assign(
        first_k_i=lambda df: df["i"].astype(str).str.slice(0, first_k_i)
    )
    print(first_k_i, test["first_k_i"].nunique(), test["first_k_i"].unique())

4 2 ['3722' '3888']
5 2 ['37222' '38886']
6 5 ['372221' '372222' '372223' '388863' '388864']
7 29 ['3722219' '3722220' '3722221' '3722222' '3722223' '3722224' '3722225'
 '3722226' '3722227' '3722228' '3722229' '3722230' '3722231' '3722232'
 '3722233' '3888636' '3888637' '3888638' '3888639' '3888640' '3888641'
 '3888642' '3888643' '3888644' '3888645' '3888646' '3888647' '3888648'
 '3888649']
8 270 ['37222199' '37222200' '37222201' '37222202' '37222203' '37222204'
 '37222205' '37222206' '37222207' '37222208' '37222209' '37222210'
 '37222211' '37222212' '37222213' '37222214' '37222215' '37222216'
 '37222217' '37222218' '37222219' '37222220' '37222221' '37222222'
 '37222223' '37222224' '37222225' '37222226' '37222227' '37222228'
 '37222229' '37222230' '37222231' '37222232' '37222233' '37222234'
 '37222235' '37222236' '37222237' '37222238' '37222239' '37222240'
 '37222241' '37222242' '37222243' '37222244' '37222245' '37222246'
 '37222247' '37222248' '37222249' '37222250' '37222251' '3722225

In [23]:
df[df.result_last_6 == ",".join([str(x) for x in program[-6:]])]

,i,result_len,result,result_last_9,result_last_8,result_last_7,result_last_6,result_last_5,result_last_4,result_last_3,result_last_2,result_first_3,result_first_4,result_first_5,result_first_6,result_first_7,result_first_8,result_first_9
1132557,37221260473399,16,"7,7,5,1,3,5,1,1,1,3,5,5,0,3,3,0","1,1,3,5,5,0,3,3,0","1,3,5,5,0,3,3,0","3,5,5,0,3,3,0","5,5,0,3,3,0","5,0,3,3,0","0,3,3,0","3,3,0","3,0","7,7,5","7,7,5,1","7,7,5,1,3","7,7,5,1,3,5","7,7,5,1,3,5,1","7,7,5,1,3,5,1,1","7,7,5,1,3,5,1,1,1"
1132558,37221260669346,16,"1,5,3,7,3,1,0,1,1,3,5,5,0,3,3,0","1,1,3,5,5,0,3,3,0","1,3,5,5,0,3,3,0","3,5,5,0,3,3,0","5,5,0,3,3,0","5,0,3,3,0","0,3,3,0","3,3,0","3,0","1,5,3","1,5,3,7","1,5,3,7,3","1,5,3,7,3,1","1,5,3,7,3,1,0","1,5,3,7,3,1,0,1","1,5,3,7,3,1,0,1,1"
1132559,37221260865292,16,"1,4,7,6,1,5,1,1,1,3,5,5,0,3,3,0","1,1,3,5,5,0,3,3,0","1,3,5,5,0,3,3,0","3,5,5,0,3,3,0","5,5,0,3,3,0","5,0,3,3,0","0,3,3,0","3,3,0","3,0","1,4,7","1,4,7,6","1,4,7,6,1","1,4,7,6,1,5","1,4,7,6,1,5,1","1,4,7,6,1,5,1,1","1,4,7,6,1,5,1,1,1"
1132560,37221261061239,16,"5,7,6,6,1,6,1,1,1,3,5,5,0,3,3,0","1,1,3,5,5,0,3,3,0","1,3,5,5,0,3,3,0","3,5,5,0,3,3,0","5,5,0,3,3,0","5,0,3,3,0","0,3,3,0","3,3,0","3,0","5,7,6","5,7,6,6","5,7,6,6,1","5,7,6,6,1,6","5,7,6,6,1,6,1","5,7,6,6,1,6,1,1","5,7,6,6,1,6,1,1,1"
1132561,37221261257185,16,"4,1,3,1,1,5,3,1,1,3,5,5,0,3,3,0","1,1,3,5,5,0,3,3,0","1,3,5,5,0,3,3,0","3,5,5,0,3,3,0","5,5,0,3,3,0","5,0,3,3,0","0,3,3,0","3,3,0","3,0","4,1,3","4,1,3,1","4,1,3,1,1","4,1,3,1,1,5","4,1,3,1,1,5,3","4,1,3,1,1,5,3,1","4,1,3,1,1,5,3,1,1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9447022,38886633060824,16,"7,7,1,3,3,1,2,1,1,3,5,5,0,3,3,0","1,1,3,5,5,0,3,3,0","1,3,5,5,0,3,3,0","3,5,5,0,3,3,0","5,5,0,3,3,0","5,0,3,3,0","0,3,3,0","3,3,0","3,0","7,7,1","7,7,1,3","7,7,1,3,3","7,7,1,3,3,1","7,7,1,3,3,1,2","7,7,1,3,3,1,2,1","7,7,1,3,3,1,2,1,1"
9447023,38886633265538,16,"1,5,1,1,2,2,3,1,1,3,5,5,0,3,3,0","1,1,3,5,5,0,3,3,0","1,3,5,5,0,3,3,0","3,5,5,0,3,3,0","5,5,0,3,3,0","5,0,3,3,0","0,3,3,0","3,3,0","3,0","1,5,1","1,5,1,1","1,5,1,1,2","1,5,1,1,2,2","1,5,1,1,2,2,3","1,5,1,1,2,2,3,1","1,5,1,1,2,2,3,1,1"
9447024,38886633470251,16,"7,2,2,7,2,1,0,1,1,3,5,5,0,3,3,0","1,1,3,5,5,0,3,3,0","1,3,5,5,0,3,3,0","3,5,5,0,3,3,0","5,5,0,3,3,0","5,0,3,3,0","0,3,3,0","3,3,0","3,0","7,2,2","7,2,2,7","7,2,2,7,2","7,2,2,7,2,1","7,2,2,7,2,1,0","7,2,2,7,2,1,0,1","7,2,2,7,2,1,0,1,1"
9447025,38886633674965,16,"5,1,3,4,1,7,1,1,1,3,5,5,0,3,3,0","1,1,3,5,5,0,3,3,0","1,3,5,5,0,3,3,0","3,5,5,0,3,3,0","5,5,0,3,3,0","5,0,3,3,0","0,3,3,0","3,3,0","3,0","5,1,3","5,1,3,4","5,1,3,4,1","5,1,3,4,1,7","5,1,3,4,1,7,1","5,1,3,4,1,7,1,1","5,1,3,4,1,7,1,1,1"


In [24]:
def generate_numbers(first_k, k, total_len):
    # Calculate how many digits remain after the prefix
    remaining = total_len - k
    # The total count of possible suffixes is 10**remaining
    max_suffix = 10**remaining

    # Sort the prefixes as integers to ensure correct ascending order
    sorted_prefixes = sorted(first_k, key=int)

    # Generate numbers by prefix
    for prefix_str in sorted_prefixes:
        prefix_val = int(prefix_str) * (10**remaining)
        for suffix in range(max_suffix):
            yield prefix_val + suffix


# Example usage:
k = 6
total_len = 14
remaining = total_len - k
first_k = ["372221", "372222", "372223", "388863", "388864"]

gen = generate_numbers(first_k, k, total_len)

tup_program = tuple(program)

# Fetch the first 20 results to illustrate:
for i, val in tqdm(enumerate(gen), total=10**remaining * len(first_k)):
    new_registers = registers.copy()
    new_registers[A] = val
    result = simulate(new_registers, program)
    result_len = len(result)

    assert result_len == 16

    if result == tup_program:
        print(val)
        break

 35%|███▍      | 173957364/500000000 [42:54<1:20:25, 67562.91it/s]

37222273957364


In [25]:
val

37222273957364